In [ ]:
from edgar_prelim import *
from edgar_query import *
from edgar_title import *
import re
from bs4 import BeautifulSoup
import pandas as pd
%load_ext autoreload
%autoreload 2
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
BAC='0000070858'
C='0000831001'
STI = '0000750556'
JPM = '0000019617'
WFC = '0000072971'
GS = '0000886982'
COF = '0000927628'
MS = '0000895421'
USB = '0000036104'
PNC = '0000713676'
BK='0001390777'
SCHW='0000316709'
STT='0000093751'
BBT='0000092230'
UBSH='0000883948'

In [ ]:
d=date(2012, 1, 20)
df = extract_prelim_statements('0001403475', start=d, end=d)
df

In [ ]:
d = date(2019, 1, 15)
filings = query_edgar_for_filings(JPM, "8-K", start=d, end=d, require_xbrl=False)
report = query_edgar_for_submission_text(filings[1].href)
submission = load_submission(report.href)
tables = list(parse_tables(submission))
items_from_tables(tables)

In [ ]:
bs = BeautifulSoup(submission.documents[-1].text, features='lxml')

In [ ]:
tables = bs.find_all('table')

In [ ]:
re.search(        fr'^{UNAUDITED}(?:(?:quarterly|condensed|select(?:ed)?)\s+)*consolidated\s+'
        fr'(?:(?:select(?:ed)?|key)\s+)*'
        r'(?:financial|operating|earnings|capital)?(?:\s+and\s+other)?'
        r'(?:\s+(?:results|metrics|highlights?|summary|information|data))+', 'CONSOLIDATED FINANCIAL HIGHLIGHTS'.lower())

In [ ]:
    table_matches = find_tag_text_matches(
        [tables[2]],
        title_inclusion_patterns,,
        max_tables=2)


In [ ]:
list(table_matches)

In [ ]:
[t.title for t in tables]

In [ ]:
units_from_table(tables[0])

In [ ]:
re.search(prelim_items[-2].name_pattern, 'Book value per common share  GAAP (a)(b)', flags=re.IGNORECASE)

In [ ]:
prelim_items[-2].name_pattern

In [ ]:
re.search(r'([1-4])\s*q(?:tr)?\s*((?:20)?\s*\d\s*\d)', 'QUARTERLY TRENDS 3Q11'.lower())

In [ ]:
%%prun
tables = list(parse_tables(submission))
items_from_tables(tables)

In [ ]:
bs = BeautifulSoup(submission.documents[-1].text, features='lxml')

In [ ]:
tables = bs.find_all('table')

In [ ]:
for t in tables:
    title = title_from_table_tag(t)
    if title:
        print(title[0])

In [ ]:
for t in tables:
   print(t.df[0])

In [ ]:
item_value_from_table(tables[4].df.iloc[:, [0, 1]], prelim_items[-1].name_pattern, 1)

In [ ]:
max_fiscal_period(tables[4].df)

In [ ]:
discard_prelim(UBSH, date(2018, 11, 16))

In [ ]:
force_reload_prelim('0001005817',  date(2013, 4, 26))

In [ ]:
force_reload_prelim('0001411974',  date(2009, 4, 28))

In [ ]:
d=date(2010, 4, 21)
filings = query_edgar_for_filings(STI, "8-K", start=d, end=d, require_xbrl=False)
report = query_edgar_for_submission_text(filings[0].href)
submission = load_submission(report.href)

In [ ]:
table_tags = BeautifulSoup(submission.documents[1].text, features='lxml').find_all('table')

In [ ]:
title_from_table_tag(table_tags[18])

In [ ]:
table_tags = list(filter(lambda x: title_from_table_tag(x) is not None, table_tags))

In [ ]:
pd.read_html(table_tags[0].prettify(), flavor='html5lib')[0].pipe(drop_single_value_rows) \
    .pipe(drop_null_rows_and_columns) \
    .fillna(value='') \
    .pipe(collapse_identical_rows) \
    .pipe(collapse_identical_columns) \
    .pipe(drop_bogus_header_rows)  \
    .pipe(collapse_header_rows) \
    .pipe(collapse_columns_with_identical_header, start_col=0)

In [ ]:
parse_fiscal_period('QTD 2011')

In [ ]:
pattern = (
    r'^(?:unaudited\s+)?(?:(?:selected|key)\s+)?(?:quarterly\s+)?'
    r'(?:financial|operating|income\s+statement)\s+'
    fr'(?:highlights?|information|results|data)$'
)
re.search(pattern, 'FINANCIAL HIGHLIGHTS', flags=re.IGNORECASE)

In [ ]:
parse_value('$48.33 	')

In [ ]:
re.search('\w', 'Interest and dividend income')

In [ ]:
widget = query_prelims_for_override('0001005817', filing_date='2016-01-28', item=None)
widget

In [ ]:
widget.save()

In [ ]:
cik_df = pd.read_sql("select * from cik", prelim_engine)

In [ ]:
cik_df.head()

In [ ]:
cik_df.to_csv("../src/resources/cik.csv")

In [ ]:
%pwd